In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Done together with @matrodriguez

In [ ]:
import numpy as np 
import pandas as pd 
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sklearn.preprocessing as skp
import sklearn.decomposition as skd
import sklearn.manifold as skm
import sklearn.cluster as skc
import scipy.cluster.hierarchy as scich 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import sklearn.model_selection as skms 
import sklearn.preprocessing as skpp 
import sklearn.metrics as skm 
import sklearn.pipeline as skpl 
import sklearn.tree as skt 
import sklearn.neighbors as skn 
import sklearn.svm as skv 
import sklearn.ensemble as ske
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
from math import isnan
from sklearn.cluster import KMeans
import pyclustering
from sklearn_extra.cluster import KMedoids
from sklearn.datasets import make_blobs
from mpl_toolkits import mplot3d
import plotly.express as px
import kmedoids
import econml
from sklearn.linear_model import TheilSenRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [ ]:
p = pd.read_csv("/kaggle/input/pisa-results-2000-2022-economics-and-education/economics_and_education_dataset_CSV.csv")
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
#p.columns.to_numpy()
p = p.drop('index_code', axis = 1)
p = p.drop('time', axis = 1)
p = p.drop('sex', axis = 1)
p

In [ ]:
p.columns.to_numpy()

In [ ]:
#dictionary:
country_prefix = p.country.unique()
countries_dict = {elem : pd.DataFrame() for elem in country_prefix}
for key in countries_dict.keys():
    countries_dict[key] = p[:][p.country == key]
    
for country in country_prefix:
    for column in countries_dict[country].select_dtypes(include='number'):
        median_country = countries_dict[country][column].median()
        countries_dict[country][column] = countries_dict[country][column].fillna(median_country)


In [ ]:
df_joineds = pd.concat(countries_dict.values())
df_joineds.dropna(how = 'any', inplace = True)
df_joineds.drop('country',axis = 1, inplace = True)
df_joineds

In [ ]:
y = df_joineds[['rating']]
x = df_joineds[['expenditure_on _education_pct_gdp', 'mortality_rate_infant',
       'gini_index', 'inflation_consumer_prices',
       'intentional_homicides', 'unemployment',
       'gross_fixed_capital_formation', 'population_density',
       'suicide_mortality_rate', 'tax_revenue',
       'taxes_on_income_profits_capital',
       'alcohol_consumption_per_capita',
       'government_health_expenditure_pct_gdp',
       'urban_population_pct_total']]


In [ ]:
corr = df_joineds.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
Q1 = df_joineds['rating'].quantile(0.25)
Q3 = df_joineds['rating'].quantile(0.75)
IQR = Q3 - Q1
((df_joineds['rating'] < (Q1 - 1.5 * IQR)) | (df_joineds['rating'] > (Q3 + 1.5 * IQR))).sum()

In [ ]:
x.columns.to_numpy()

In [ ]:
# huber reg
import statsmodels.api as sm
import statsmodels as smd
import statsmodels.formula.api as smf

In [ ]:
huber_t = sm.RLM(endog = y, 
                   exog= x, M=sm.robust.norms.HuberT())
hub_results = huber_t.fit()
print(hub_results.params)
print(
    hub_results.summary(
        yname="y", xname=["var_%d" % i for i in range(len(hub_results.params))]
    )
)

In [ ]:
#r2:
r2_wls = sm.WLS(huber_t.endog, huber_t.exog, weights=huber_t.fit().weights).fit().rsquared
r2_wls

In [ ]:
#Quantile Regression
quanti = smf.quantreg('y ~ x' ,df_joineds).fit(q=0.7)
 
# view model summary
print(quanti.summary())

In [ ]:
# quasi-experiment:
scaler = skp.StandardScaler( )
p_norm = pd.DataFrame( scaler.fit_transform( df_joineds.values ), index = df_joineds.index, columns = df_joineds.columns)
p_norm = p_norm.dropna(how='any')
p_norm

In [ ]:
pca = skd.PCA(svd_solver='randomized', random_state=42)
# fiting PCA on the dataset
pca.fit(p_norm)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
fig = plt.figure(figsize = (12,8))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
pca_5c_obj = skd.PCA( n_components = 6) 
pca_d = pca_5c_obj.fit_transform( p_norm.values ) 

pca_5c_obj.explained_variance_ratio_
sum( pca_5c_obj.explained_variance_ratio_ )

In [ ]:
from sklearn.decomposition import IncrementalPCA
pca_f_d = IncrementalPCA(n_components=6)
df_pca_ = pca_f_d.fit_transform(p_norm)
df_pca_.shape
pc_d = np.transpose(df_pca_)

In [ ]:
corrmat = np.corrcoef(pc_d)
plt.figure(figsize = (20,10))
sns.heatmap(corrmat,annot = True)
plt.show()

In [ ]:
pcr_df = pd.DataFrame({'PC1':pc_d[0],'PC2':pc_d[1],'PC3':pc_d[2],'PC4':pc_d[3],'PC5':pc_d[4],'PC6':pc_d[5]})
pcr_df.shape 
pcr_df

In [ ]:
#clusterization:
ssd = []
for num_clusters in list(range(1,10)):
    model_clus = KMedoids(n_clusters = num_clusters, max_iter=50)
    model_clus.fit(pcr_df)
    ssd.append(model_clus.inertia_)

plt.plot(ssd)
plt.show()

In [ ]:
k_m = KMedoids(n_clusters=8, random_state=0)
k_m.fit(pcr_df)

In [ ]:
k_m.labels_

In [ ]:
k_ml = pd.DataFrame(k_m.labels_)
k_ml.columns = ['Cluster_md']
k_ml

In [ ]:
df_joineds.reset_index(drop=True, inplace=True)
k_m = pd.concat([k_ml,df_joineds],axis = 1)
k_m

In [ ]:
sns.scatterplot(x= k_m['Cluster_md'],y='rating',data=k_m)

In [ ]:
c0 =k_m.query('Cluster_md == 0')
c1 =k_m.query('Cluster_md == 1')
c2 =k_m.query('Cluster_md == 2')
c3 =k_m.query('Cluster_md == 3')
c4 =k_m.query('Cluster_md == 4')
c5 =k_m.query('Cluster_md == 5')
c6 =k_m.query('Cluster_md == 6')
c7 =k_m.query('Cluster_md == 7')

In [ ]:
dd= pd.concat([c0,c1],ignore_index=True)
dd.columns.to_numpy() 

In [ ]:
#from scipy.stats import ttest_ind
#_, p = ttest_ind(dd.expenditure_on _education_pct_gdp, df_joineds.expenditure_on _education_pct_gdp)
#print(f'p={p:.3f}')

# interpret
#alpha = 0.05  # significance level
#if p > alpha:
  #  print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
#else:
 #   print('different distributions/different group mean (reject H0)')

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.rating, k_m.rating)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')
    

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.mortality_rate_infant, k_m.mortality_rate_infant)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')
    
from scipy.stats import ttest_ind
_, p = ttest_ind(dd.gini_index, k_m.gini_index)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')
from scipy.stats import ttest_ind
_, p = ttest_ind(dd.gdp_per_capita_ppp, k_m.gdp_per_capita_ppp)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.inflation_consumer_prices, k_m.inflation_consumer_prices)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')
from scipy.stats import ttest_ind
_, p = ttest_ind(dd.intentional_homicides, k_m.intentional_homicides)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.unemployment, k_m.unemployment)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.gross_fixed_capital_formation, k_m.gross_fixed_capital_formation)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.population_density, k_m.population_density)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.suicide_mortality_rate, k_m.suicide_mortality_rate)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.alcohol_consumption_per_capita, k_m.alcohol_consumption_per_capita)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.taxes_on_income_profits_capital, k_m.taxes_on_income_profits_capital)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.government_health_expenditure_pct_gdp, k_m.government_health_expenditure_pct_gdp)
print(f'p={p:.3f}')

# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

from scipy.stats import ttest_ind
_, p = ttest_ind(dd.urban_population_pct_total, k_m.urban_population_pct_total)
print(f'p={p:.3f}')
# interpret
alpha = 0.05  # significance level
if p > alpha:
    print('same distributions/same group mean (fail to reject H0 - we do not have enough evidence to reject H0)')
else:
    print('different distributions/different group mean (reject H0)')

In [ ]:
#Logit:
x_ = dd[['expenditure_on _education_pct_gdp',
       'gini_index', 'gdp_per_capita_ppp',
       'inflation_consumer_prices', 
       'unemployment', 'gross_fixed_capital_formation',
       'suicide_mortality_rate', 'tax_revenue',
       'taxes_on_income_profits_capital',
       'alcohol_consumption_per_capita',
      'urban_population_pct_total' ]]

y_ = dd[['Cluster_md']]

In [ ]:
l =   'urban_population_pct_total'
Q1 = dd[l].quantile(0.25)
Q3 = dd[l].quantile(0.75)
IQR = Q3 - Q1
((dd[l] < (Q1 - 1.5 * IQR)) | (dd[l] > (Q3 + 1.5 * IQR))).sum()

In [ ]:
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm
model= sm.Logit(y_,x_)
co =model.fit().params
coe = pd.DataFrame(co)
coe.columns = ['coeff']
exp_coeff =   np.exp(coe)/(np.exp(coe) + 1) 
exp_coeff = pd.DataFrame(exp_coeff)
exp_coeff.columns = ['exp_coeff']
exp_coeff

In [ ]:
results = model.fit()
results_summary = results.summary()
results_summary

In [ ]:
results_as_html = results_summary.tables[1].as_html()
df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
df_results['exp_coeff'] = exp_coeff
df_results